<a href="https://colab.research.google.com/github/CSheppardCodes/Scholastic-Study-of-Machine-Learning/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(1) We are going to use the following dataset for this exercise:
https://archive.ics.uci.edu/ml/datasets/Health+News+in+Twitter
Follow the “Data Folder” link and unzip the given file. You will file a folder containing tweets
that contain links to various news sources e.g. the file “usnewshealth.txt” contains tweets that
refer to articles published in US News. You have to choose one such file and proceed.

In [ ]:
import random as rd
import re
import math
import string
import pandas as pd

#df = pd.read_csv("https://raw.githubusercontent.com/CSheppardCodes/MLDatasetsUCI/main/asg3/bbchealth.txt")
#dataurl = df.values.tolist()

(2) Perform the following pre-processing steps:
- Remove the tweet id and timestamp
- Remove any word that starts with the symbol @ e.g. @AnnaMedaris
- Remove any hashtag symbols e.g. convert #depression to depression
- Remove any URL
- Convert every word to lowercase

In [ ]:
def pre_process_tweets(url):

    f = open(url, "r", encoding="utf8")
    tweets = list(f)
    list_of_tweets = []

    for i in range(len(tweets)):


        tweets[i] = tweets[i].strip('\n')                                             # remove \n from the end after every sentence
        tweets[i] = tweets[i][50:]                                                    # Remove the tweet id and timestamp
        tweets[i] = " ".join(filter(lambda x: x[0] != '@', tweets[i].split()))        # Remove any word that starts with the symbol @
        tweets[i] = re.sub(r"http\S+", "", tweets[i])                                 # Remove URL
        tweets[i] = re.sub(r"www\S+", "", tweets[i])                                  # Remove URL

        # remove colons from the end of the sentences (if any) after removing url
        tweets[i] = tweets[i].strip()
        tweet_len = len(tweets[i])
        if tweet_len > 0:
            if tweets[i][len(tweets[i]) - 1] == ':':
                tweets[i] = tweets[i][:len(tweets[i]) - 1]

        tweets[i] = tweets[i].replace('#', '')                                        # Remove any hash-tags symbols
        tweets[i] = tweets[i].lower()                                                 # Convert every word to lowercase
        tweets[i] = tweets[i].translate(str.maketrans('', '', string.punctuation))    # remove punctuations
        tweets[i] = " ".join(tweets[i].split())                                       # trim extra spaces
        list_of_tweets.append(tweets[i].split(' '))                                   # convert each tweet from string type to as list<string> using " " as a delimiter

    f.close()
    return list_of_tweets

Perform K-means clustering on the resulting tweets using at least 5 different values of K and report your results in the format below
K is the number of clusters and mi is the centroid of the ith cluster.

In [ ]:

def k_means(tweets, k=4, max_iterations=50):

    centroids = []

    # initialization, assign random tweets as centroids
    count = 0
    hash_map = dict()
    while count < k:
        random_tweet_idx = rd.randint(0, len(tweets) - 1)
        if random_tweet_idx not in hash_map:
            count += 1
            hash_map[random_tweet_idx] = True
            centroids.append(tweets[random_tweet_idx])

    iter_count = 0
    prev_centroids = []

    # run the iterations until not converged or until the max iteration in not reached
    while (is_converged(prev_centroids, centroids)) == False and (iter_count < max_iterations):

        #print("running iteration " + str(iter_count))

        # assignment, assign tweets to the closest centroids
        clusters = assign_cluster(tweets, centroids)

        # to check if k-means converges, keep track of prev_centroids
        prev_centroids = centroids

        # update, update centroid based on clusters formed
        centroids = update_centroids(clusters)
        iter_count = iter_count + 1

    if (iter_count == max_iterations):
        print("max iterations reached, K means not converged")
    else:
        print("")

    sse = compute_SSE(clusters)

    return clusters, sse


def is_converged(prev_centroid, new_centroids):

    # false if lengths are not equal
    if len(prev_centroid) != len(new_centroids):
        return False

    # iterate over each entry of clusters and check if they are same
    for c in range(len(new_centroids)):
        if " ".join(new_centroids[c]) != " ".join(prev_centroid[c]):
            return False

    return True


def assign_cluster(tweets, centroids):

    clusters = dict()

    # for every tweet iterate each centroid and assign closest centroid to a it
    for t in range(len(tweets)):
        min_dis = math.inf
        cluster_idx = -1;
        for c in range(len(centroids)):
            dis = getDistance(centroids[c], tweets[t])
            # look for a closest centroid for a tweet

            if centroids[c] == tweets[t]:
                # print("tweet and centroid are equal with c: " + str(c) + ", t" + str(t))
                cluster_idx = c
                min_dis = 0
                break

            if dis < min_dis:
                cluster_idx = c
                min_dis = dis

        # randomise the centroid assignment to a tweet if nothing is common
        if min_dis == 1:
            cluster_idx = rd.randint(0, len(centroids) - 1)

        # assign the closest centroid to a tweet
        clusters.setdefault(cluster_idx, []).append([tweets[t]])
        # print("tweet t: " + str(t) + " is assigned to cluster c: " + str(cluster_idx))
        # add the tweet distance from its closest centroid to compute sse in the end
        last_tweet_idx = len(clusters.setdefault(cluster_idx, [])) - 1
        clusters.setdefault(cluster_idx, [])[last_tweet_idx].append(min_dis)

    return clusters


def update_centroids(clusters):

    centroids = []

    # iterate each cluster and check for a tweet with closest distance sum with all other tweets in the same cluster
    # select that tweet as the centroid for the cluster
    for c in range(len(clusters)):
        min_dis_sum = math.inf
        centroid_idx = -1

        # to avoid redundant calculations
        min_dis_dp = []

        for t1 in range(len(clusters[c])):
            min_dis_dp.append([])
            dis_sum = 0
            # get distances sum for every of tweet t1 with every tweet t2 in a same cluster
            for t2 in range(len(clusters[c])):
                if t1 != t2:
                    if t2 < t1:
                        dis = min_dis_dp[t2][t1]
                    else:
                        dis = getDistance(clusters[c][t1][0], clusters[c][t2][0])

                    min_dis_dp[t1].append(dis)
                    dis_sum += dis
                else:
                    min_dis_dp[t1].append(0)

            # select the tweet with the minimum distances sum as the centroid for the cluster
            if dis_sum < min_dis_sum:
                min_dis_sum = dis_sum
                centroid_idx = t1

        # append the selected tweet to the centroid list
        centroids.append(clusters[c][centroid_idx][0])

    return centroids


def getDistance(tweet1, tweet2):

    # get the intersection
    intersection = set(tweet1).intersection(tweet2)

    # get the union
    union = set().union(tweet1, tweet2)

    # return the jaccard distance
    return 1 - (len(intersection) / len(union))


def compute_SSE(clusters):

    sse = 0
    # iterate every cluster 'c', compute SSE as the sum of square of distances of the tweet from it's centroid
    for c in range(len(clusters)):
        for t in range(len(clusters[c])):
            sse = sse + (clusters[c][t][1] * clusters[c][t][1])

    return sse



In [ ]:

if __name__ == '__main__':

    datatxt = 'bbchealth.txt'
    tweets = pre_process_tweets(datatxt)

    # default number of experiments to be performed
    kplus = 5

    # default value of K for K-means
    k = 3

    # for every experiment 'e', run K-means
    for e in range(kplus):

        print("No. " + str((e + 1)))
        print("K = " + str(k))

        clusters, sse = k_means(tweets, k)

        # for every cluster 'c', print size of each cluster
        for c in range(len(clusters)):
            print(str(c+1) + ": ", str(len(clusters[c])) + " tweets")


        print("SSE : " + str(sse))
        print('\n')

        # increment k after every experiment
        k = k + 2

No. 1
K = 3

1:  1412 tweets
2:  1507 tweets
3:  1010 tweets
SSE : 3391.3437963066544


No. 2
K = 5

1:  1379 tweets
2:  619 tweets
3:  880 tweets
4:  593 tweets
5:  458 tweets
SSE : 3323.0554131000804


No. 3
K = 7

1:  526 tweets
2:  470 tweets
3:  743 tweets
4:  229 tweets
5:  333 tweets
6:  882 tweets
7:  746 tweets
SSE : 3266.6444096414034


No. 4
K = 9

1:  776 tweets
2:  311 tweets
3:  497 tweets
4:  412 tweets
5:  699 tweets
6:  356 tweets
7:  428 tweets
8:  185 tweets
9:  265 tweets
SSE : 3214.4781595903637


No. 5
K = 11

1:  222 tweets
2:  890 tweets
3:  152 tweets
4:  174 tweets
5:  362 tweets
6:  557 tweets
7:  249 tweets
8:  371 tweets
9:  321 tweets
10:  455 tweets
11:  176 tweets
SSE : 3236.2937325783814


